## Machine Learning methods to predict 

In [21]:
#Importing data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

#Importing data
metadata = pd.read_csv('metadata_labeled.csv')
metadata = metadata.drop('Label',axis=1)
multiqc = pd.read_csv('multiqc_data.csv')
label= pd.read_csv('labels.csv',sep='\t')

In [22]:
#Merge metad and execution dataframe on Run
df = pd.merge(metadata, multiqc, on='Run', how='inner')
#extract for M0 from label dataframe along with Run column
label= label.iloc[:,[0,1]]
#rename the last column as label
label.columns = ['Run','Label']
#merge metadata and label dataframe on Run
metadata = pd.merge(metadata,label, on='Run', how='inner')
#Merge df and label dataframe on Run
df = pd.merge(df, label, on='Run', how='inner')

In [23]:
#Merging multiqc and metadata
merged = pd.merge(metadata, multiqc,on='Run')
#Removing columns with no data
merged = merged.dropna(axis=1, how='all')
#Removing columns with only one unique value
merged = merged.loc[:,merged.apply(pd.Series.nunique) != 1]
#Removing columns with more than 40% missing values
merged = merged.loc[:, merged.isnull().mean() < .4]
#Removing columns with more than 40% zeros
merged = merged.loc[:, (merged == 0).mean() < .4]
#Remove non-numeric columns
merged = merged.select_dtypes(include=[np.number])
#put Label column at the end in merged dataframe
merged = merged[[c for c in merged if c not in ['Label']] + ['Label']]
merged.head()
#Replace nan with mean of that column
merged = merged.fillna(merged.mean())
#Extract features which has 0.5 correlation with Label
# correlation = merged.corr()
# correlation = correlation['Label'].sort_values(ascending=False)
# features = correlation.index[abs(correlation)>0.7]
# features = features.drop('Label')
# df=merged[features]
# #add labels to the dataframe
# df['Label'] = merged['Label']
df = merged
#Remove size_MB from df
# df = df.drop('size_MB', axis=1)
#Remove size_MB from features
# features = features.drop('size_MB')
#extract only size_MB and Label in df
# df = df[['size_MB','Label']]
df.shape
# features

(101, 386)

In [26]:
# #Vislualize the data
# import matplotlib.pyplot as plt
# import seaborn as sns
# #pearson correlation between the variables of merged_new
# # merged_new2=merged_new.drop(columns=['Run'])
# correlation = data.corr()
# plt.figure(figsize=(50,50))
# sns.heatmap(correlation, vmax=1, square=True, annot=True, cmap='cubehelix')
# plt.title('Correlation between different features')
# plt.show()
df.to_csv('Processed_data.csv', index=False)

In [117]:
# X= df.drop('Label', axis=1)
# y = df['Label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# #Feature scaling
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# #Train the model
# # Define the models
# models = [
#     ('Linear Regression', LinearRegression()),
#     ('Gradient Boosting', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0)),
#     ('Lasso Regression', Lasso()),
#     ('Ridge Regression', Ridge()),
#     ('SVM', SVR()),
#     ('Random Forest', RandomForestRegressor()),
#     ('Decision Tree', DecisionTreeRegressor()),
#     ('Neural Network', MLPRegressor(max_iter=50000)),
#     ('XGBoost', XGBRegressor())
# ]

# # Define a function to perform cross-validation and calculate metrics
# # def cross_val_and_evaluate(model, X, y):
# #     scores_mae = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=10)
# #     scores_mse = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10)
# #     scores_r2 = cross_val_score(model, X, y, scoring='r2', cv=10)
# #     mean_mae = -1 * scores_mae.mean()
# #     mean_mse = -1 * scores_mse.mean()
# #     mean_r2 = scores_r2.mean()
# #     return mean_mae, mean_mse, mean_r2

# def cross_val_and_evaluate(model, X, y):
#     scores_mae = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=10, error_score='raise')
#     scores_mse = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10, error_score='raise')
#     scores_r2 = cross_val_score(model, X, y, scoring='r2', cv=10, error_score='raise')
#     mean_mae = -1 * scores_mae.mean()
#     mean_mse = -1 * scores_mse.mean()
#     mean_r2 = scores_r2.mean()
#     return mean_mae, mean_mse, mean_r2

# # Define a function to display the metrics in a table
# def display_results(results):
#     print(f"{'Model':<20}{'MAE':<10}{'MSE':<10}{'R2':<10}")
#     for name, metrics in results.items():
#         mean_mae, mean_mse, mean_r2 = metrics
#         print(f"{name:<20}{mean_mae:<10.2f}{mean_mse:<10.2f}{mean_r2:<10.2f}")
# # Train each model, perform cross-validation, calculate the metrics, and add the results to the table
# results = {}
# for name, model in models:
#     results[name] = cross_val_and_evaluate(model, X, y)

# #sort the results based on mean R2 value
# results_sorted = dict(sorted(results.items(), key=lambda item: item[1][2], reverse=True))
# display_results(results_sorted)


In [25]:
#Model comparison
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define the models
models = [
    ('Linear Regression', LinearRegression()),
    ('Lasso Regression', Lasso()),
    ('Ridge Regression', Ridge()),
    ('SVM', SVR()),
    ('Random Forest', RandomForestRegressor()),
    ('Decision Tree', DecisionTreeRegressor()),
    ('Neural Network', MLPRegressor(max_iter=50000)),
    ('XGBoost', XGBRegressor())
]

X=metadata[['spots', 'bases', 'spots_with_mates', 'avgLength', 'size_MB', 'InsertSize']]
#Merge X and df to get the final dataframe
X=pd.merge(X, df, left_index=True, right_index=True)
#Remove duplicate columns from X and Label column
X = X.loc[:,~X.columns.duplicated()]
#Encode the Label column
labelencoder = LabelEncoder()
X['Label'] = labelencoder.fit_transform(X['Label'])
y=X['Label']
X=X.drop('Label', axis=1)

# X= df.drop('Label', axis=1)
# y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#Scale the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Define a function to train a model and calculate metrics
def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    try:
        feature_importance = model.feature_importances_
        # print('Feature importance:', feature_importance)
        
    except AttributeError:
        feature_importance = None
    return mae, mse, r2, feature_importance

# Define a function to display the metrics in a table
#dipslay feature importance in decimal by two decimal places
def display_results(results):
    print(f"{'Model':<20}{'MAE':<10}{'MSE':<10}{'R2':<10}")
    for name, metrics in results.items():
        mae, mse, r2, feature_importance = metrics
        if feature_importance is not None:
            feature_importance = [f"{value:.2f}" for value in feature_importance]
            feature_importance = ' '.join(feature_importance)
        else:
            feature_importance = ''
        print(f"{name:<20}{mae:<10.2f}{mse:<10.2f}{r2:<10.2f}")

# Train each model, calculate the metrics, and add the results to the table
results = {}
for name, model in models:
    results[name] = train_and_evaluate(model, X_train, y_train, X_test, y_test)

# Display the table
# display_results(results)

#sort the results based on R2 value
results_sorted = dict(sorted(results.items(), key=lambda item: item[1][2], reverse=True))
display_results(results_sorted)

'''['spots', 'bases', 'spots_with_mates', 'avgLength', 'size_MB',
       'InsertSize', 'psl_0', 'None_mqc-generalstats-fastqc-total_sequences',
       'Unique Reads', 'psgc_41', 'psgc_45', 'psgc_49', 'psgc_42', 'psgc_40',
       'psgc_46', 'psgc_44', 'psgc_48', 'psgc_39', 'psgc_53', 'psgc_50',
       'psgc_43', 'psgc_38', 'psgc_47', 'psgc_52', 'pss_1', 'psgc_37',
       'psgc_51', 'psgc_54', 'psgc_36', 'psgc_35', 'Duplicate Reads',
       'psgc_34', 'psgc_55', 'psgc_56', 'psgc_33', 'psgc_57', 'pss_0',
       'psgc_32', 'psgc_31', 'psgc_30', 'psgc_58', 'psgc_0', 'psgc_29',
       'pss_13']'''

Model               MAE       MSE       R2        
Random Forest       4.57      34.15     0.95      
Decision Tree       5.10      39.38     0.95      
XGBoost             5.05      43.68     0.94      
Lasso Regression    10.69     218.21    0.70      
Ridge Regression    14.02     478.80    0.35      
SVM                 20.27     574.65    0.21      
Neural Network      15.33     638.83    0.13      
Linear Regression   17.22     801.31    -0.09     


"['spots', 'bases', 'spots_with_mates', 'avgLength', 'size_MB',\n       'InsertSize', 'psl_0', 'None_mqc-generalstats-fastqc-total_sequences',\n       'Unique Reads', 'psgc_41', 'psgc_45', 'psgc_49', 'psgc_42', 'psgc_40',\n       'psgc_46', 'psgc_44', 'psgc_48', 'psgc_39', 'psgc_53', 'psgc_50',\n       'psgc_43', 'psgc_38', 'psgc_47', 'psgc_52', 'pss_1', 'psgc_37',\n       'psgc_51', 'psgc_54', 'psgc_36', 'psgc_35', 'Duplicate Reads',\n       'psgc_34', 'psgc_55', 'psgc_56', 'psgc_33', 'psgc_57', 'pss_0',\n       'psgc_32', 'psgc_31', 'psgc_30', 'psgc_58', 'psgc_0', 'psgc_29',\n       'pss_13']"

In [228]:
#extract top 10 importance features for best model above
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
feature_importance = model.feature_importances_
features = X.columns
features_importance = dict(zip(features, feature_importance))
features_importance = dict(sorted(features_importance.items(), key=lambda item: item[1], reverse=True))
features_importance = dict(list(features_importance.items())[:10])
features_importance



{'size_MB_y': 0.706482802096668,
 'psgcp_6': 0.08051976936244708,
 'psgcp_64': 0.07678833612043111,
 'psgcp_74': 0.059680048898120695,
 'bases_y': 0.01442709178638129,
 'psgc_92': 0.013738603204160189,
 'psgc_53': 0.011326673033439176,
 'pbsq_16': 0.006251641710128301,
 'pss_17': 0.0051474393667803806,
 'pbsq_44': 0.003749839512502308}

In [229]:
#Use the top 10 features to train the model and calculate the metrics
top_features = X[['size_MB_y', 'psgcp_6', 'psgcp_64', 'psgcp_74', 'bases_y', 'psgc_92', 'psgc_53', 'pbsq_16', 'pss_17', 'pbsq_44']]
#extract only top_features and Label 
top_features['Label'] = y
#predict using top_features
X=top_features.drop('Label', axis=1)

In [ ]:
#Save results_sorted to a csv file and keep the rows name 
results_sorted_df = pd.DataFrame(results_sorted)
#rename rows as Models, MAE, MSE, R2, Feature Importance
results_sorted_df = results_sorted_df.rename(index={0: 'MAE', 1: 'MSE', 2: 'R2', 3: 'Feature Importance'})
results_sorted_df.to_csv('Model_comparison.csv')

75.38333333333334
